In [12]:
import pandas as pd
import numpy as np
from unittest import result
from pulp import *

In [13]:
df_nc = pd.read_csv('nc.txt')
df_dpe = pd.read_csv('dpe.txt')
df_dist = pd.read_csv('dist.txt',sep=';')


In [14]:
# vetor demanda
demanda = np.array(df_nc['pessoas'])

# distância do no a empresa
dpe = np.array(df_dpe['dist'])
a = [0]
b =np.concatenate((a,dpe))
# matriz das distâncias
df_dist.fillna(0,inplace=True)
df_dist.insert(0,'0',dpe,True)
df_dist = df_dist.T
df_dist.insert(0,'0',b,True)
dist = np.array(df_dist)
# custo por Km

m =40
n =31
m_k= np.concatenate([np.repeat(3.2,10),np.repeat(2.9,30)])
Q_k= np.concatenate([np.repeat(48,10),np.repeat(16,30)])
F_k= np.concatenate([np.repeat(15000/44,10),np.repeat(4000/44,30)])

c = np.zeros((m,n,n))
for k in np.arange(m):
    if k < 10:
        c[k] = m_k[k]*dist
    else:
        c[k] = m_k[k]*dist
c = np.array(c)

In [15]:
prob = LpProblem("Problema_trafego",LpMinimize)

x = LpVariable.dicts("matriz dos x",[(j,i,k) for j in np.arange(n) for i in np.arange(n) for k in np.arange(m)],cat='Binary')

# Objetivo do Problema

for k in range(m):
    for j in range(n):
        prob += lpSum(F_k[k]*x[(0,j,k)])
        for i in range(n):
            prob += lpSum( c[k][j][i] * x[(i,j,k)] ) 

    # Restricoes

    prob += lpSum(x[(i,j,k)]  for i in np.arange(1,n) for j in np.arange(n) ) == 1

    prob += lpSum(x[(i,j,k)]  for i in np.arange(n) for j in np.arange(1,n) ) == 1

    prob += lpSum(x[(0,j,k)]  for j in np.arange(n) ) == 1

    prob += lpSum(x[(i,0,k)]  for i in np.arange(n) ) == 1

    prob +=lpSum(x[(i,j,k)]*demanda[j] for i in range(n) for j in range(n)) <= Q_k[k]

    prob += lpSum(x[(i,j,k)]+x[(j,i,k)] for i in np.arange(1,n) for j in np.arange(1,n) ) <= 1

result = prob.solve()

In [16]:
if result == LpStatusOptimal:
    t = 0
    for k in range(m):
        for j in range(n):
            t += F_k[k]*x[(0,j,k)].value()
            for i in range(n):
                t += (c[k][j][i] * x[(i,j,k)].value() ) 
    print('O menor valor diário será: {:2.2f}'.format(t))

O menor valor diário será: 14641.60
